In [572]:
import pandas as pd
import numpy as np
import re

# 데이터 불러오고 전처리

### 출처 

* https://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do 서울특별시 버스정류소 위치정보 > 서울 열린 데이터 광장
* https://www.starbucks.co.kr/store/store_map.do 스타벅스 매장 찾기 정보 (API를 통해 크롤링하여 정보 수집)

In [1031]:
bus_raw = pd.read_csv('C:/Users/whileduck/Dropbox/python/bus.csv',encoding = 'cp949') #서울특별시 버스정류장들의 경도 위도 데이터 
starbucks_raw = pd.read_csv('C:/Users/whileduck/Dropbox/python/starbucks.csv', index_col=0) # API 로 크롤링 해온 스타벅스 매장 데이터

In [1032]:
starbucks_raw # 사용 할 데이터 탐색 

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.513931,127.020606,서울,강남구,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...,...
593,사가정역,37.579594,127.087966,서울,중랑구,서울특별시 중랑구 면목로 310,1522-3232
594,상봉역,37.596890,127.086470,서울,중랑구,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
595,묵동,37.615368,127.076633,서울,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
596,양원역,37.606654,127.106360,서울,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [1033]:
bus_raw # 내가 사용할 데이터는 정류소명과 좌표값이다.

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230
...,...,...,...,...,...
11162,124000334,25995,우성아파트,127.139326,37.550442
11163,124000333,25996,우성아파트,127.140052,37.550620
11164,124000332,25997,조일약국,127.123477,37.533686
11165,124000331,25998,성내시장,127.125410,37.536199


In [1034]:
bus = bus_raw.iloc[:,-3:] #필요 없는 데이터 제거

In [1035]:
bus       #버스 정류장이 같은 이름으로 여러 개가 있다. 아마도 방향에 따라 같은 정류장이 2개 이상 있기 때문
          # 종착역이나 순환 버스들도 있기 때문에 꼭 2개씩 반복되지는 않는듯하다.

,정류소명,좌표X,좌표Y
0,종로2가사거리,126.987786,37.569764
1,창경궁.서울대학교병원,126.996520,37.579179
2,명륜3가.성대입구,126.998290,37.582709
3,종로2가.삼일교,126.987507,37.568582
4,혜화동로터리.여운형활동터,127.001694,37.586230
...,...,...,...
11162,우성아파트,127.139326,37.550442
11163,우성아파트,127.140052,37.550620
11164,조일약국,127.123477,37.533686
11165,성내시장,127.125410,37.536199


In [1036]:
bus = bus.drop_duplicates('정류소명')
bus = bus.reset_index(drop = True)  #중복값이 있으니 제거해주자 

In [1037]:
bus.columns = ['정류소명','lot','lat']
bus = bus[['정류소명','lat','lot']]
bus #column 명과 순서 변경 

,정류소명,lat,lot
0,종로2가사거리,37.569764,126.987786
1,창경궁.서울대학교병원,37.579179,126.996520
2,명륜3가.성대입구,37.582709,126.998290
3,종로2가.삼일교,37.568582,126.987507
4,혜화동로터리.여운형활동터,37.586230,127.001694
...,...,...,...
7052,강빛초.중학교,37.574115,127.172355
7053,굽은다리역,37.544370,127.142455
7054,천호우체국.로데오거리,37.540488,127.127251
7055,조일약국,37.533686,127.123477


In [1038]:
starbucks_raw.columns = ['매장명','경도','위도','시','구','도로명 주소','전화번호'] # 함수 값에서 직관적으로 보일 수 있도록 컬럼명 변경

### 거리를 구할건데 어떤 방식으로 구할 것이냐

 * 2차원 형태인 지도에서의 거리를 구할 것이기 때문에 맨하탄 거리 방식을 사용하자 
 * 정류장 이름을 넣었을 때 정류장의 좌표값이 호출 되도록 만드는 함수 
 * 정류장과 스타벅스 간의 거리를 구하고 가장 가까운 거리의 스타벅스 매장을 호출하는 함수

### 정류장 명을 말했을 때 버스 값의 좌표가 나오는 함수

In [1039]:
def bus_location(x):
    
    '''
    버스 정류장 명을 입력하면 해당 정류장의 경도,위도가 
    호출되는 함수입니다.
    '''
     
    global bus

 
    bus_lat = bus.loc[bus['정류소명'] == x,'lat']
    bus_lot = bus.loc[bus['정류소명'] == x,'lot']

    return np.array([bus_lat,bus_lot])
        
 


In [1040]:
dobong_array = bus_location('도봉보건소')
dobong_array # 잘 작동한다

array([[ 37.65894194],
       [127.04074799]])

In [1041]:
bus_location('세상에 없는 정류장이름!') # bus 데이터 프레임에 없는 정보를 입력하면 오류가 뜨지 않고 빈 데이터 프레임이 array 형태로 변환됨 

array([], shape=(2, 0), dtype=float64)

### 경도와 위도 좌표를 입력 했을 때 거리를 구해주는 함수 


##### 스타벅스의 경도, 위도 좌표를 계산에 용이하게 array 형태로 변환 

In [1042]:
starbucks_array = np.array(starbucks_raw[['경도','위도']]).reshape(-1,2)
starbucks_array

array([[ 37.501087  , 127.043069  ],
       [ 37.510178  , 127.022223  ],
       [ 37.5139309 , 127.0206057 ],
       ...,
       [ 37.615368  , 127.076633  ],
       [ 37.60665363, 127.1063598 ],
       [ 37.60170912, 127.0784114 ]])

##### 해당 정류장의 좌표와 스타벅스매장들간의 경도,위도의 거리 계산

In [1043]:
lat_distance= abs(starbucks_array[:,0] - dobong_array[0]) # 도봉보건소 정류장의 경도와 스타벅스 데이터들의 경도의 차이 계산
lot_distance= abs(starbucks_array[:,1] - dobong_array[1]) # 도봉보건소 정류장의 위도와 스타벅스 데이터들의 위도의 차이 계산 
display(lat_distance[:10]) , display(lot_distance[:10])     # 각각의 거리를 담은 array 의 모습 

array([0.15785494, 0.14876394, 0.14501104, 0.15942494, 0.16427394,
       0.14394194, 0.13614854, 0.14879194, 0.15119194, 0.13157504])

array([0.00232101, 0.01852499, 0.02014229, 0.00925299, 0.02183501,
       0.02244801, 0.01214709, 0.01952701, 0.01990301, 0.00768699])

(None, None)

In [1044]:
distance = (lat_distance + lot_distance).reshape(-1,1) # 두 거리의 합을 distance 변수에 담아줌
                                                       # 각 값들이 독립적이라는 것을 직관적으로 이해하기 위해 1열 형태로 reshape
distance

array([[0.16017595],
       [0.16728893],
       [0.16515333],
       [0.16867793],
       [0.18610895],
       [0.16638995],
       [0.14829563],
       [0.16831895],
       [0.17109495],
       [0.13926203],
       [0.23283595],
       [0.17781293],
       [0.16328195],
       [0.15530995],
       [0.17090093],
       [0.16965895],
       [0.14600795],
       [0.16669795],
       [0.17353993],
       [0.17939295],
       [0.15248695],
       [0.14384693],
       [0.15523593],
       [0.13480495],
       [0.17638393],
       [0.13937113],
       [0.17367995],
       [0.16809035],
       [0.15188213],
       [0.16274503],
       [0.18342495],
       [0.14914993],
       [0.14142395],
       [0.17644795],
       [0.17258673],
       [0.16372395],
       [0.15490563],
       [0.15869193],
       [0.16638693],
       [0.17100393],
       [0.14539075],
       [0.16632105],
       [0.17556995],
       [0.15431089],
       [0.1807493 ],
       [0.18279095],
       [0.16670896],
       [0.165

##### 가장 거리값이 최소가 되는, 즉 가장 가까운 거리의 인덱스 값을 min_distance_idx 변수에 담아줌

In [1045]:
min_distance_idx = np.argmin(distance) #가장 거리가 최소가 되는 인덱스 호출
min_distance_idx

180

In [1046]:
starbucks_raw.loc[[min_distance_idx]] # 해당 거리의 차이가 작은 인덱스 번호에 해당되는 스타벅스의 정보 호출

,매장명,경도,위도,시,구,도로명 주소,전화번호
180,방학역,37.665312,127.043312,서울,도봉구,"서울특별시 도봉구 도봉로 684 (방학동)101,102,113,114,115호",1522-3232


### 테스트는 끝났고 함수로 만들자

In [1047]:
def find_starbucks(station):
    
    '''
    정류장 이름을 입력하면 가장 가까운 스타벅스의 정보를 
    알려주는 함수입니다.
    '''
    
    global starbucks_raw
    global bus
    try:
        location = bus_location(station)

        starbucks_lat = np.array(starbucks_raw['경도']).reshape(-1,1)
        starbucks_lot = np.array(starbucks_raw['위도']).reshape(-1,1)

        lat_distance = abs(starbucks_lat - location[0])
        lot_distance = abs(starbucks_lot - location[1])

        distance = lat_distance + lot_distance

        min_distance = np.argmin(distance)

        return starbucks_raw.iloc[[min_distance]]

    except:
        print ('버스 정류장명이 정확한지 다시 확인해주세요!')
        return bus[['정류소명']]


In [1048]:
find_starbucks('미아사거리') #잘 작동한다.

,매장명,경도,위도,시,구,도로명 주소,전화번호
298,미아이마트,37.610678,127.029921,서울,성북구,서울특별시 성북구 도봉로 17 (길음동),1522-3232


In [1049]:
find_starbucks('미아사오정') # 만약 사용자가 정류소명을 헷갈려서 bus 데이터에 없는 버스 정류소명을 입력했을 때에는 다시 확인해보도록 정류소명들을 보여줬는데 
                          # 사용자에게 7057개의 모든 데이터를 읽어보고 맞춰서 다시 검색하라고 하는 것은 비효율적일 것이라고 판단됨

버스 정류장명이 정확한지 다시 확인해주세요!


,정류소명
0,종로2가사거리
1,창경궁.서울대학교병원
2,명륜3가.성대입구
3,종로2가.삼일교
4,혜화동로터리.여운형활동터
...,...
7052,강빛초.중학교
7053,굽은다리역
7054,천호우체국.로데오거리
7055,조일약국


### 만약 사용자가 입력한 정류소 명이 bus 데이터에서 글자가 조금만 바뀐거라면 어떻게 할까?

* 최대한 사용자가 입력한 단어와 내가 가진 데이터 중 유사한 단어를 고를 수 있게 만들어야 한다. 
* 그 전에 버스 정류장 이름을 입력 했을 때 내가 가진 데이터에 있는 경우, 없는 경우 두 가지의 경우로 나눠서 함수를 짜야 한다. 
       * 기존에 만든 bus_location 함수에서 bus 데이터에 없는 변수명을 입력 했을 때 오류가 나지 않고 빈 데이터 프레임이 나온다. 
       * 그렇기 때문에 bus_location 함수에서 내가 가진 데이터에 없는 변수명을 입력 했을 때 오류를 호출하도록 만들자

* 1. 사용자가 입력한 문자와 내가 가진 데이터의 유사도를 변수에 담는다. (SequencMachter,append)
* 2. 유사도 들 중 가장 높은 값의 인덱스를 찾는다.
        - 유사도의 인덱스 값과 내가 가진 데이터들은 1:1 상응하기 때문.
* 3. 가장 높은 유사도를 가진 인덱스의 값으로 내가 가진 데이터에서 변수를 호출
* 4. 호출한 변수로 기존에 만든 함수인 find_starbucks로 돌린다.
     

# bus_location 함수에서 없는 데이터를 입력 했을 때 오류가 뜨게 수정 

* if 조건문으로 했어도 됐겠지만 함수 간 충돌이 일어 났을 때 수정에 용이하도록 try , except 를 활용
* 오류가 발생 했을 시 오류의 의미가 혼동될 수 있기 때문에 내가 지정한 오류가 뜰 수 있도록 NotImplementedError가 뜨도록 설정

In [1050]:
def bus_location(x):
    
    '''
    버스 정류장 명을 입력하면 해당 정류장의 경도,위도가 
    호출되는 함수입니다.
    '''
     
    global bus
    
    if len(bus.loc[bus['정류소명'] == x]):
        bus_lat = bus.loc[bus['정류소명'] == x,'lat']
        bus_lot = bus.loc[bus['정류소명'] == x,'lot']

        return np.array([bus_lat,bus_lot])
    
    else:
        
        raise NotImplementedError
        
 


In [1051]:
bus_location('미아사오정') # 아까와 다르게 빈 값을 호출하는 것이 아니라 설정한 오류가 뜬다

NotImplementedError: 

# 그럼 이제 데이터에 없는 정보를 입력했을 때 어떻게 가장 비슷한 문자열을 찾아야 할까

* 여러가지 방법을 구상해봤다.

##### 1. str 에서 list(str) 로 단어를 쪼개 준 다음에 in list 를 통해 부분 조건문으로 True , False 의 형식으로 바꿔 준 다음 행 방향으로 더해주는 방법
   * 1.1 입력한 데이터를 list(str) 을 통해 ['없','는','데','이','터'] 이런 식으로 쪼개주고 동일한 방식으로 bus 데이터 프레임의 정류소 명들도 쪼개줌
   * 1.2 반복문을 통해 ['없','는','데','이','터'] 의 값들을 하나 씩 꺼내서 쪼개진 bus의 정류소명들에 하나씩 부분 조건문으로 boolean 형태로 보관
   * 1.3 boolean 값들을 합해줘 가장 True 값들이 많은 변수의 인덱스로 정류소명을 결정
   * 단점 : 반복문을 두개를 중첩해서 써야해서 시간이 오래 걸릴 거 같다고 판단
    
    
##### 2. set(bus['정류소명']) - set([사용자가 입력한 데이터] ) 를 빼 준다음 가장 문자열의 길이가 짧은 값을 가장 유사한 값이라 가정하는 방법
   * 가장 직관적인 방법일 수 있지만 이렇게 될 경우 버스 정류소 명이 극단적으로 짧은 데이터가 있을 시 어떤 데이터를 입력하든 극단적으로 짧은 이름을 가진 데이터가 호출됨
   
### 3. 파이썬 내장 모듈인 difflib 의 SequencMatcher.ratio() 이용
   * 가장 쉬운 방법으로 SequencMatcher(None,bus['정류소명],'사용자가 입력한 데이터').ratio()로 두 문자열의 유사도를 구한 후 가장 높은 유사도의 인덱스로 정류소명을 정함



##### 1. 사용자가 입력한 단어와 bus['정류소명']의 값들간의 유사도 계산 

In [1052]:
station = '미아사오정' #원래 정류소명은 미아사거리지만 사용자가 헷갈려서 미아육거리라는 단어를 입력했을 시를 가정 

ratio_list = []
for i,var in enumerate(bus['정류소명']): #정류소명의 모든 값들을 하나씩 순회하도록 enumerate
        ratio_list.append((SequenceMatcher(None,station,var).ratio())) #사용자가 입력한 단어와 순회하는 bus['정류소명']의 값들의 유사도 계산

display(ratio_list[:10])
print('유사도를 담은 리스트와 bus["정류소명의 길이는"]',len(ratio_list),len(bus['정류소명'])) # 사용자가 입력한 단어와 bus['정류소명']의 값들과의 유사도가 담긴 리스트를 생성     
                                                                            # 유사도가 담긴 리스트와 bus['정류소명']의 길이는 같다. 즉, 둘의 인덱스는 1:1 상응함

[0.16666666666666666,
 0.0,
 0.0,
 0.0,
 0.0,
 0.16666666666666666,
 0.11764705882352941,
 0.0,
 0.0,
 0.0]

유사도를 담은 리스트와 bus["정류소명의 길이는"] 7057 7057


In [1053]:
print('유사도가 가장 높은 값은 :',max(ratio_list)) # 유사도가 담긴 리스트에서 가장 높은 값을 찾음

max_similar_idx = ratio_list.index(max(ratio_list)) # 가장 높은 값의 인덱스 번호를 변수에 저장
max_similar_name = bus['정류소명'].iloc[max_similar_idx] # 해당 인덱스의 정류소 명을 변수에 저장 

print('유사도가 가장 높은 값의 인덱스 넘버는:',max_similar_idx)
print('해당 인덱스의 정류소 명 값은',max_similar_name)

유사도가 가장 높은 값은 : 0.6
유사도가 가장 높은 값의 인덱스 넘버는: 1688
해당 인덱스의 정류소 명 값은 미아사거리


##### 사용자가 입력한 데이터는 미아육거리였지만 가장 유사도가 높은 단어를 bus 데이터에서 꺼내와 미아 사거리라는 값을 호출해냈음 

In [1054]:
display(bus_location(max_similar_name)) #미아사거리의 경도와 위도 값이 나오는 모습 

array([[ 37.6101564 ],
       [127.03018161]])

# 해당 과정을 토대로 함수를 수정하여 완성

In [1055]:
def find_starbucks(station):
    
    
    '''
    정류장 이름을 입력하면 가장 가까운 스타벅스의 정보를 
    알려주는 함수입니다.
    
    만약 입력한 정류장 명이 가지고 있는 데이터에 없을 경우 
    가장 유사도가 높은 정류장명으로 수정하여 검색합니다.
    '''
    
    global starbucks_raw
    global bus
    
    
    
    
    try:
        
        
        
        location = bus_location(station)
        starbucks_lat = np.array(starbucks_raw['경도']).reshape(-1,1)
        starbucks_lot = np.array(starbucks_raw['위도']).reshape(-1,1)

        lat_distance = abs(starbucks_lat - location[0])
        lot_distance = abs(starbucks_lot - location[1])

        distance = lat_distance + lot_distance

        min_distance = np.argmin(distance)
        
        print(station,'과 가장 가까운 스타벅스의 정보는')

        return starbucks_raw.iloc[[min_distance]]
    
    except: 
        
        print('정류소명이 정확하지 않아 유사한 정류소명을 찾겠습니다.')
        
        ratio_list = []
        for i,var in enumerate(bus['정류소명']):
            ratio_list.append((SequenceMatcher(None,station,var).ratio()))
            
            
        max_similar_idx = ratio_list.index(max(ratio_list))
        max_similar_name = bus['정류소명'].iloc[max_similar_idx]
            
        print('당신이 입력한 정류소 명:',station,
              '유사한 정류소 명 :',max_similar_name,'유사도 : ',max(ratio_list) * 100,'%')
            
        location = bus_location(max_similar_name)
        
        # 이 밑에서부터 거리를 계산하여 스타벅스 정보룰 호출하는 값은 위와 같음
        
        starbucks_lat = np.array(starbucks_raw['경도']).reshape(-1,1)
        starbucks_lot = np.array(starbucks_raw['위도']).reshape(-1,1)

        lat_distance = abs(starbucks_lat - location[0])
        lot_distance = abs(starbucks_lot - location[1])

        distance = lat_distance + lot_distance

        min_distance = np.argmin(distance)

        return starbucks_raw.iloc[[min_distance]]


In [1056]:
find_starbucks('미아사거리') # 정확한 정보명을 입력했을 때의 결과

미아사거리 과 가장 가까운 스타벅스의 정보는


,매장명,경도,위도,시,구,도로명 주소,전화번호
298,미아이마트,37.610678,127.029921,서울,성북구,서울특별시 성북구 도봉로 17 (길음동),1522-3232


In [1057]:
find_starbucks('미아사오정') # 부정확한 정보명을 입력했을 때 유사도가 높은 단어로 검색한 결과 

정류소명이 정확하지 않아 유사한 정류소명을 찾겠습니다.
당신이 입력한 정류소 명: 미아사오정 유사한 정류소 명 : 미아사거리 유사도 :  60.0 %


,매장명,경도,위도,시,구,도로명 주소,전화번호
298,미아이마트,37.610678,127.029921,서울,성북구,서울특별시 성북구 도봉로 17 (길음동),1522-3232
